In [5]:
# STAP1

import os
import cv2
import numpy as np
import random
from sklearn.model_selection import train_test_split
import pickle

IMAGE_SIZE = (64, 64)

def load_images_from_folder(folder, label):
    data = []
    for filename in os.listdir(folder):
        filepath = os.path.join(folder, filename)
        image = cv2.imread(filepath)
        if image is not None:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # OpenCV gebruikt BGR
            image = cv2.resize(image, IMAGE_SIZE)
            data.append((image, label))
    return data

# faces_train = load_images_from_folder(
#     r"C:\Users\vanho\OneDrive\data_science\WORKDIR\2024_VDO_DS\notebooks\20 Machine learning\13 Neural nets\OPDRACHT\Face_Recognition_DS\Face_Recognition\faces", 
#     1)
# 
# faces_test = load_images_from_folder(
#     r"C:\Users\vanho\OneDrive\data_science\WORKDIR\2024_VDO_DS\notebooks\20 Machine learning\13 Neural nets\OPDRACHT\Face_Recognition_DS\Face_Recognition\faces_test", 
#     1)
# 
# no_faces_train = load_images_from_folder(
#     r"C:\Users\vanho\OneDrive\data_science\WORKDIR\2024_VDO_DS\notebooks\20 Machine learning\13 Neural nets\OPDRACHT\Face_Recognition_DS\Face_Recognition\no_faces\linnaeus\Linnaeus 5 128X128\train\all", 
#     0)
# 
# no_faces_test = load_images_from_folder(
#     r"C:\Users\vanho\OneDrive\data_science\WORKDIR\2024_VDO_DS\notebooks\20 Machine learning\13 Neural nets\OPDRACHT\Face_Recognition_DS\Face_Recognition\no_faces\linnaeus\Linnaeus 5 128X128\test\all", 
#     0)

### Paths' Sali
faces_train = load_images_from_folder(
    r"C:\Users\salia\firstProject\GPU\13 Neural nets\OPDRACHT\DeepLearning\Face_Recognition\faces", 
    1)

faces_test = load_images_from_folder(
    r"C:\Users\salia\firstProject\GPU\13 Neural nets\OPDRACHT\DeepLearning\Face_Recognition\faces_test", 
    1)

no_faces_train = load_images_from_folder(
    r"C:\Users\salia\firstProject\GPU\13 Neural nets\OPDRACHT\DeepLearning\Face_Recognition\no_faces\linnaeus\Linnaeus 5 128X128\train\all", 
    0)

no_faces_test = load_images_from_folder(
    r"C:\Users\salia\firstProject\GPU\13 Neural nets\OPDRACHT\DeepLearning\Face_Recognition\no_faces\linnaeus\Linnaeus 5 128X128\test\all", 
    0)

train_data = faces_train + no_faces_train
test_data = faces_test + no_faces_test

random.shuffle(train_data)
random.shuffle(test_data)

X_train = np.array([item[0] for item in train_data])
y_train = np.array([item[1] for item in train_data])

X_test = np.array([item[0] for item in test_data])
y_test = np.array([item[1] for item in test_data])

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

X_train = X_train / 255.0
X_val = X_val / 255.0
X_test = X_test / 255.0

with open("train_data.pkl", "wb") as f:
    pickle.dump((X_train, y_train), f)

with open("val_data.pkl", "wb") as f:
    pickle.dump((X_val, y_val), f)

with open("test_data.pkl", "wb") as f:
    pickle.dump((X_test, y_test), f)

print("Dataset resized, gelabeld, gesplitst en opgeslagen.")


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\salia\\firstProject\\GPU\\13 Neural nets\\OPDRACHT\\DeepLearning\\Face_Recognition\\faces'

In [6]:
 pip install scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# STAP2

import os
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image

with open("train_data.pkl", "rb") as f:
    X_train, y_train = pickle.load(f)

with open("val_data.pkl", "rb") as f:
    X_val, y_val = pickle.load(f)

with open("test_data.pkl", "rb") as f:
    X_test, y_test = pickle.load(f)

print(f"Train data: {X_train.shape}, Validation data: {X_val.shape}, Test data: {X_test.shape}")

# MODEL
model = models.Sequential([
    layers.InputLayer(input_shape=(64, 64, 3)),  # De input size (64x64)

    # 1e Convolutionele laag + MaxPooling
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),

    # 2e Convolutionele laag + MaxPooling
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),

    # 3e Convolutionele laag + MaxPooling
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),

    # Flatten de output
    layers.Flatten(),

    # Fully connected layer
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),  # Dropout voor regularisatie

    # Output layer voor binaire classificatie (gezicht vs niet-gezicht)
    layers.Dense(1, activation='sigmoid')  # Sigmoid voor binaire classificatie
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_val, y_val),
    verbose=1
)

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f"Test accuracy: {test_acc:.4f}")

import matplotlib.pyplot as plt

# Accuracy plotten
plt.plot(history.history['accuracy'], label='Train acc')
plt.plot(history.history['val_accuracy'], label='Val acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Model Accuracy')
plt.show()

# Loss plotten
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Model Loss')
plt.show()


model.save("face_classifier_model.h5")
print("Model opgeslagen als 'face_classifier_model.h5'.")












In [7]:
#STEP 3:

import cv2
import numpy as np
import os
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

# Load trained model
model = load_model('face_classifier_model.h5')  # adjust path if needed

# Folder with group images
test_folder = 'Face_Recognition/faces_test'

# Sliding window params
window_size = 64
stride = 16
threshold = 0.8

# Loop through .png images
for filename in os.listdir(test_folder):
    if filename.endswith('.png'):
        img_path = os.path.join(test_folder, filename)
        image = cv2.imread(img_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        h, w, _ = image.shape

        boxes = []

        for y in range(0, h - window_size, stride):
            for x in range(0, w - window_size, stride):
                crop = image[y:y+window_size, x:x+window_size]
                resized = cv2.resize(crop, (64, 64)) / 255.0
                input_img = np.expand_dims(resized, axis=0)

                score = model.predict(input_img, verbose=0)[0][0]

                if score > threshold:
                    boxes.append([x, y, x + window_size, y + window_size])

        # Draw detected boxes
        for box in boxes:
            x1, y1, x2, y2 = box
            cv2.rectangle(image_rgb, (x1, y1), (x2, y2), (255, 0, 0), 2)

        # Show image
        plt.figure(figsize=(8, 8))
        plt.imshow(image_rgb)
        plt.title(f"Detections - {filename}")
        plt.axis('off')
        plt.show()

SyntaxError: invalid syntax (102320844.py, line 1)